In [12]:
import torch
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [13]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel
import math
import Model_Import_6
from torch import optim
import random
from sklearn.model_selection import train_test_split
from transformers import AdamW
import numpy as np
# from transformers import WarmupLinearSchedule
import gc


In [14]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

42.314694656
6.43825664
6.421954048
0.016302592


In [15]:
def load_stuff(context_tensors, generator_logits, text_ids_loc):
    context = torch.load(context_tensors)
    logits = torch.load(generator_logits)
    text_ids = torch.load(text_ids_loc)
    return context, logits, text_ids

In [16]:
R_neg_embeds, neg_logits, neg_token_ids = load_stuff('R_neg_embeds.pt', 'neg_logits.pt', 'neg_token_ids.pt')
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [17]:
R_pos_embeds, pos_logits, pos_token_ids = load_stuff('R_pos_embeds.pt', 'pos_logits.pt', 'pos_token_ids.pt')
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [55]:
def one_style_generate(prompt, tokenizer, SAT_model, GPT_transformer, context_to_sample, num_samples = 100, num_tokens_to_generate = 1, sen_to_generate = 10):
  #Put models in eval mode
    SAT_model.eval()
    GPT_transformer.eval()
    
    for sent in range(sen_to_generate):
        random_context_samples = random.sample(context_to_sample, num_samples)
        model_context_input =  torch.stack(random_context_samples, dim = 0).to(device)
        expierment_list = []
        sentences = []
        # tokenize prompt
        current_tokenization = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

        for generation in range(num_tokens_to_generate):
            # put tokenized prompt through GPT_transformer to get GPT Logits
            GPT_logits = GPT_transformer(current_tokenization).last_hidden_state.squeeze()

            # put model_context_input and the GPT Logits into SAT model
            adjusted_output = SAT_model(model_context_input, GPT_logits)

            # Funtional softmax the SAT model output
            SM_adjusted_output = torch.nn.functional.softmax(adjusted_output, dim = 1)

            # argmax to get predicted token
            # predicted_tokens = torch.argmax(SM_adjusted_output, dim =1)

            # get topk tokens
            top_predicted_tokens = torch.topk(SM_adjusted_output[-1], 3, dim =0).indices
            top_predicted_tokens_prob = torch.topk(SM_adjusted_output[-1], 3, dim =0).values
            top_predicted_tokens_numpy = top_predicted_tokens.cpu().detach().numpy()
            top_predicted_tokens_prob_numpy = top_predicted_tokens_prob.cpu().detach().numpy()
            token_prob_sum = np.sum(top_predicted_tokens_prob_numpy)
            token_distribution = top_predicted_tokens_prob_numpy/token_prob_sum
            
            predicted_token =  torch.from_numpy(np.array(np.random.choice(top_predicted_tokens_numpy, p = token_distribution))).to(device)
            
            # print(top_predicted_tokens)
             # tok_k_predicted_words = tokenizer.decode(top_predicted_tokens, skip_special_tokens=True)
#             decoded_tokens = []
#             for token_k in top_predicted_tokens:
#                 decoded_tokens.append(tokenizer.decode(token_k, skip_special_tokens=True))
                
#             top_predicted_tokens_prob_cpu = top_predicted_tokens_prob.cpu()
            del top_predicted_tokens_prob
            # expierment_list.append((top_predicted_tokens_prob_cpu, decoded_tokens))


                
            #dif way of getting top predicted
            # predicted_token = top_predicted_tokens[0]
            # print(current_tokenization[-1][0])
            # print(predicted_token)
            # if predicted_token == 247 or predicted_token == current_tokenization[-1][0]:
            if predicted_token == 247:
                print("WEIRD TOKEN PREDICTED")
                predicted_token = top_predicted_tokens[1]

            # print(torch.amax(SM_adjusted_output[-1]))

            # print(predicted_tokens[-1].unsqueeze(0).unsqueeze(0))
            # print(current_tokenization)

            current_tokenization = torch.cat((current_tokenization, predicted_token.unsqueeze(0).unsqueeze(0)), 1)
            del predicted_token
            gc.collect()
            # print(current_tokenization)

        # decode
        sentence = []
        for i, beam in enumerate(current_tokenization):
            # print(f"{i}: {tokenizer.decode(beam)}")
            # print(f"{i}: {current_tokenization}")
            #print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
            sentence.append(tokenizer.decode(beam, skip_special_tokens=True))
        sentences.append(sentence)
        del current_tokenization
        del random_context_samples
        del model_context_input
        gc.collect()
        # for prediction in expierment_list:
        #     print(prediction)
        print("\n\n")
    return sentences
    # for i, beam in enumerate(predicted_tokens):
    #     # if i == 0:
    #     #   continue
    #     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)} token_id: {beam}")


In [19]:

# all_embeds = R_neg_embeds+R_pos_embeds
style_list_neg = []
style_list_pos = []
for embed in R_neg_embeds:
    style_list_neg.append(0)
for embed in R_pos_embeds:
    style_list_pos.append(1)
    
style_list = style_list_neg + style_list_pos


In [20]:
logits_train, logits_test, token_ids_train, token_ids_test, style_train, style_test = train_test_split(neg_logits+pos_logits, neg_token_ids+pos_token_ids, style_list, test_size=0.2, random_state=42)

In [21]:
R_embeds_neg_train, R_embeds_neg_test = train_test_split(R_neg_embeds, test_size=0.2, random_state=42)
R_embeds_pos_train, R_embeds_pos_test = train_test_split(R_pos_embeds, test_size=0.2, random_state=42)

In [22]:
def train_two_styles_w_dev(model, optimizer, pos_context_embeds_list, neg_context_embeds_list, logits_list, token_ids_list, style_list_train, dev_logits, dev_pos_context, dev_neg_context, dev_token_ids, dev_style, epochs, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list, style_list_train))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list, style_list_train = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            if style_list_train[example] == 0:
                context_embeds_list = neg_context_embeds_list
            else:
                context_embeds_list = pos_context_embeds_list
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        model.eval()
        CELoss_dev = nn.CrossEntropyLoss()
        dev_loss_acum = 0
        for dev_example in range(len(dev_logits)):
            if style_list_train[dev_example] == 0:
                dev_context = dev_neg_context
            else:
                dev_context = dev_pos_context
            random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
            stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
            
            dev_network_output = model(stacked_context_sample_dev.to(device), dev_logits[dev_example].to(device))
            
            shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
            shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous().to(device)
            dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
            dev_loss_acum += dev_loss.item()
        full_dev_loss = dev_loss_acum / len(dev_logits)
        
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [41]:
num_heads = 8

In [42]:
prompt = "The movie"

In [43]:
pytorch_stacked_two_cross= Model_Import_6.MultiHeadModel_PyTorch_Stacked(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer = optim.RAdam(pytorch_stacked_two_cross.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [44]:
train_two_styles_w_dev(pytorch_stacked_two_cross, neg_optimizer, R_embeds_pos_train, R_embeds_neg_train, logits_train, token_ids_train, style_train, logits_test, R_embeds_pos_test, R_embeds_neg_test, token_ids_test, style_test, 10, num_samples = 100)

....................Epoch: 0, Epoch Examples: 19999
TRAIN LOSS: 3.261791944503784
DEV LOSS: 3.24163070807457
----------------------------------------
....................Epoch: 1, Epoch Examples: 19999
TRAIN LOSS: 3.2213380336761475
DEV LOSS: 3.2333539516448973
----------------------------------------
....................Epoch: 2, Epoch Examples: 19999
TRAIN LOSS: 3.2006990909576416
DEV LOSS: 3.230662031984329
----------------------------------------
....................Epoch: 3, Epoch Examples: 19999
TRAIN LOSS: 3.1808905601501465
DEV LOSS: 3.2353980966329576
----------------------------------------
....................Epoch: 4, Epoch Examples: 19999
TRAIN LOSS: 3.162254810333252
DEV LOSS: 3.2333726744413376
----------------------------------------
....................Epoch: 5, Epoch Examples: 19999
TRAIN LOSS: 3.144083023071289
DEV LOSS: 3.2388961870670316
----------------------------------------
....................Epoch: 6, Epoch Examples: 19999
TRAIN LOSS: 3.1267900466918945
DEV L

In [45]:
pytorch_basic = Model_Import_6.MultiHeadModel_PyTorch(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer_basic = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [46]:
train_two_styles_w_dev(pytorch_basic, neg_optimizer_basic, R_embeds_pos_train, R_embeds_neg_train, logits_train, token_ids_train, style_train, logits_test, R_embeds_pos_test, R_embeds_neg_test, token_ids_test, style_test, 10, num_samples = 10)

....................Epoch: 0, Epoch Examples: 19999
TRAIN LOSS: 3.2572386264801025
DEV LOSS: 3.237956048643589
----------------------------------------
....................Epoch: 1, Epoch Examples: 19999
TRAIN LOSS: 3.2205159664154053
DEV LOSS: 3.2310818600654603
----------------------------------------
....................Epoch: 2, Epoch Examples: 19999
TRAIN LOSS: 3.199735164642334
DEV LOSS: 3.233080428123474
----------------------------------------
....................Epoch: 3, Epoch Examples: 19999
TRAIN LOSS: 3.1804263591766357
DEV LOSS: 3.2303701306462287
----------------------------------------
....................Epoch: 4, Epoch Examples: 19999
TRAIN LOSS: 3.1619551181793213
DEV LOSS: 3.231329568481445
----------------------------------------
....................Epoch: 5, Epoch Examples: 19999
TRAIN LOSS: 3.1449155807495117
DEV LOSS: 3.234548204445839
----------------------------------------
....................Epoch: 6, Epoch Examples: 19999
TRAIN LOSS: 3.129037857055664
DEV L

In [34]:
pytorch_stacked_two_alt = Model_Import_6.MultiHeadModel_PyTorch_Stacked_One_Alt(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], heads = num_heads, attention_dim = int(neg_logits[0].shape[1])).to(device) #
# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
neg_optimizer_basic = optim.RAdam(pytorch_stacked_two_alt.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [ ]:
train_two_styles_w_dev(pytorch_stacked_two_alt, neg_optimizer_basic, R_embeds_pos_train, R_embeds_neg_train, logits_train, token_ids_train, style_train, logits_test, R_embeds_pos_test, R_embeds_neg_test, token_ids_test, style_test, 10, num_samples = 10)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_alt, Model_Import_6.head_transformer, R_embeds_neg_test, num_samples = 10, num_tokens_to_generate = 50, sen_to_generate = 5)

In [ ]:
one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_alt, Model_Import_6.head_transformer, R_embeds_pos_test, num_samples = 10, num_tokens_to_generate = 50, sen_to_generate = 5)

In [47]:
neg_SAT_generations = one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_embeds_neg_test, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 100)

0: The movie was so bad it was good. It was so terrible that the acting was so good that the movie was actually good. It was so bad it was good, it was so bad that the acting wasn't so good that the movie was actually good.



0: The movie was so bad that I actually turned it off after the first half hour. The acting was bad. The script was bad. The editing was bad. The directing was bad and the editing was so bad that it was almost like watching paint dry. I



0: The movie was not the worst movie I have ever seen but it was the first one that I thought was bad. I was very disappointed. The acting was bad. The plot was bad. It was a movie that I didn't want to watch. The only



0: The movie was so bad that it was banned in Germany. I was so disappointed that the movie was so bad that it was banned in Germany, and I was so disappointed that the movie was so bad that it was banned in Germany. I was so disappointed that



0: The movie was so bad that I couldn't believe it was even made

KeyboardInterrupt: 

In [ ]:
pos_SAT_generations = one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_embeds_pos_test, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 100)

In [ ]:
neg_SAT_generations_basic = one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_embeds_neg_test, num_samples = 10, num_tokens_to_generate = 50, sen_to_generate = 5)

In [ ]:
pos_SAT_generations_basic = one_style_generate(prompt, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, R_embeds_pos_test, num_samples = 10, num_tokens_to_generate = 50, sen_to_generate = 5)

In [48]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [50]:
import nltk
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [51]:
# ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
import re
import string
from collections import Counter
from collections import defaultdict
gatsby_first_words_list = []
for sen_g in gatsby_sen_list:
    sen_g = re.sub(r'^\s*|\s\s*', ' ', sen_g).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_g = filter(lambda x: x in printable, sen_g)
    sen_g = ''.join(filter(lambda x: x in printable, sen_g))
    sen_g_list = sen_g.split()
    #print(sen_g_list[0:3])
    gatsby_first_words_list.append(sen_g_list[0:3])
    
origin_first_words_list = []  
for sen_o in origin_sen_list:
    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_o = filter(lambda x: x in printable, sen_o)
    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
    sen_o_list = sen_o.split()
    # print(sen_o_list[0:3])
    origin_first_words_list.append(sen_o_list[0:3]) 
    
initial_unigrams_gatsby = defaultdict(int)
initial_bigrams_gatsby = defaultdict(int)
initial_trigrams_gatsby = defaultdict(int)
initial_unigrams_origin = defaultdict(int)
initial_bigrams_origin = defaultdict(int)
initial_trigrams_origin = defaultdict(int)
for first_words in gatsby_first_words_list:
    initial_unigrams_gatsby[first_words[0]] += 1
    initial_bigrams_gatsby[' '.join(first_words[:2])] += 1
    initial_trigrams_gatsby[' '.join(first_words)] += 1
    
for first_words in origin_first_words_list:
    initial_unigrams_origin[first_words[0]] += 1
    initial_bigrams_origin[' '.join(first_words[:2])] += 1
    initial_trigrams_origin[' '.join(first_words)] += 1
    
unigram_overlap = set(initial_unigrams_gatsby.keys())
unigram_overlap = unigram_overlap.intersection(initial_unigrams_origin.keys())

bigram_overlap = set(initial_bigrams_gatsby.keys())
bigram_overlap = bigram_overlap.intersection(initial_bigrams_origin.keys())
   
trigram_overlap = set(initial_trigrams_gatsby.keys())
trigram_overlap = trigram_overlap.intersection(initial_trigrams_origin.keys())


# print(f"unigram_overlap: {str(unigram_overlap)}")
# print("\n")
print(f"bigram_overlap: {str(bigram_overlap)}")
print("\n")
# print(f"trigram_overlap: {str(trigram_overlap)}")

bigram_list = list(bigram_overlap)
    
# combine the list

# combined_g_o_list = gatsby_first_words_list + origin_first_words_list

# mini_sen_list = []
# for begin in combined_g_o_list:
#     mini_sen_list.append(' '.join(begin))
    
# # ref https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python
# bigrams = zip(mini_sen_list, mini_sen_list[1:])
# counts = Counter(bigrams)
# print(counts.most_common())

bigram_overlap: {'We could', 'I think', 'The only', 'How do', 'In two', 'I remember', 'Let the', 'And when', 'I cannot', 'The day', 'A new', 'But the', 'No one', 'All these', 'I feel', 'But there', 'It is', 'But we', 'But in', 'Not that', 'By the', 'See how', 'The practical', 'Were all', 'In the', 'I must', 'And as', 'The little', 'A few', 'Of course', 'He has', 'If we', 'How the', 'With the', 'At a', 'In fact,', 'He always', 'As soon', 'I believe', 'I see', 'But it', 'I will', 'Even if', 'Look at', 'There must', 'They are', 'One of', 'A little', 'A man', 'This was', 'I am', 'To the', 'But with', 'But I', 'The eyes', 'And what', 'I could', 'At this', 'The most', 'I should', 'At first', 'In my', 'The whole', 'As we', 'Some little', 'Even when', 'I would', 'I know', 'The fact', 'On a', 'But what', 'A large', 'After a', 'The old', 'And I', 'The truth', 'It might', 'But how', 'If I', 'As I', 'A white', 'I was', 'It seems', 'Now I', 'Let us', 'After the', 'The flowers', 'I have', 'And we', 

In [52]:
lst = random.sample(bigram_list,100)

In [56]:
neg_SAT_generations = []
for prompt_ in lst:
    neg_SAT_generations.append(one_style_generate(prompt_, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_embeds_neg_test, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 1)[0])

In [65]:
pos_SAT_generations = []
for prompt_ in lst:
    pos_SAT_generations.append(one_style_generate(prompt_, Model_Import_6.tokenizer, pytorch_stacked_two_cross, Model_Import_6.head_transformer, R_embeds_pos_test, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 1)[0])

In [66]:
def clean_strings(strings):
    cleaned_strings = []
    #print(strings[:20])
    for sen in strings:
        #print(sen)
        sen = re.sub(r"\\", "", sen[0])    # bizarrely, each string is inside of a list containing nothing but that string
        sen = re.sub(r"\n", " ", sen)
        sen = filter(lambda x: x in printable, sen)
        sen = ''.join(filter(lambda x: x in printable, sen))
        cleaned_strings.append(sen)
    #print(cleaned_strings[:20])
    
    return cleaned_strings

In [67]:
clean_pos_SAT_generations = clean_strings(pos_SAT_generations)
clean_neg_SAT_generations = clean_strings(neg_SAT_generations)

In [68]:
all_seqs = []
for seq in clean_pos_SAT_generations:
    all_seqs.append(('POS_SAT', seq))
for seq in clean_neg_SAT_generations:
    all_seqs.append(('NEG_SAT', seq))


In [69]:
df = pd.DataFrame(all_seqs, columns = ['Source', 'Sequence'])

df.to_csv('pos_neg_sat.csv', encoding='utf-8', index = False)